# Source
파이썬 클린코드 2장

프로퍼티

* 프로퍼티는 객체의 어떤 속성에 대한 접근을 __제어__하려는 경우 사용한다.(파이써닉한 코드다.)

* 자바와 같은 다른 프로그래밍 언어에서는 접근 메서드인 게터(getter)와 세터(setter)를 만드나 파이썬에선 프로퍼티를 사용한다.

* 아래의 예는 사용자가 등록하는 정보에 잘못된 정보가 입력되지 않도록 보호하려는 예이다.

In [12]:
# 상황 : 이메일을 입력받을 때 이메일의 형식에 맞게 입력하게끔 하고싶다.
#        이메일이라는 속성을 형식에 맞게 보호 하고싶다.
#        이메일이라는 속성에 접근할 때 형식에 맞지 않으면 에러를 발생시키는 제어를 하고싶다.


import re

EMAIL_FORMAT = re.compile(r"[^@]+@[^@]+[^@]+") # ^@ == @를 제외한 모든 문자열


def is_valid_email(potentially_valid_email: str):
    return re.match(EMAIL_FORMAT, potentially_valid_email) is not None


class User:
    def __init__(self, username):
        self.username = username
        self._email = None 
        # 밑줄은 email이 private으로 사용될 것이므로 외부에서 접근하면 안 된다는 뜻이다.
        # 밑줄 자체에 어떤 효과가 있는 것은 아니나 컨벤션으로 사용된다고 생각하자.

    
    # private 속성인 email 값을 반환하는 @property 메서드
    @property
    def email(self):
        return self._email

    # <user>.email = <new_email>이 실행될 때 호출되는 코드
    # <new_email>은 파라미터가 된다.
    @email.setter
    def email(self, new_email):
        if not is_valid_email(new_email):
            raise ValueError(f"유효한 이메일이 아니므로 {new_email} 값을 사용할 수 없습니다.")
        self._email = new_email

In [13]:
student = User("Daniel")
student.email = "daniel@" # @email.setter 메서드를 잘 호출한 것을 볼 수 있다.

ValueError: ignored

In [16]:
student.email = "DanielPidy@gamil.com"
student.email # student._email도 접근은 가능하지만 이렇게 하진 말자.(사생활 보호!!)

'DanielPidy@gamil.com'

> 프로퍼티는 명령-쿼리 분리 원칙(CC08)을 따르기 위한 좋은 방법이라고 한다.
    
    명령-쿼리 분리 원칙:
        객체의 메서드는 둘 중 하나의 역할만 수행해야한다.(둘 다 수행하면 안됨)
        1. 무언가의 **상태를 변경**하는 커맨드
        2. 무언가의 **값을 반환**하는 쿼리
> 즉, 객체의 메서드가 무언가를 하는 동시에 질문에 대답하기 위한 상태를 반환한다면 이는 동시에 두 가지 작업을 하고 있는 것 => 명령-쿼리 분리 원칙 위배.

> 메서드 이름에 따라 실제 코드가 무엇을 하는지 혼란스럽고 이해하기 어려운 경우가 있음

    ex)
        set_email이라는 메서드를
        if self.set_email("a@b.com") 처럼 사용한다면 이 코드는 무엇을 의미하는지 불분명해짐.
        (이메일을 "a@b.com"으로 정하겠다는 것인가?,
         이메일이 "a@b.com"으로 설정되어 있는지 확인하는 것인가?,
         이메일 값을 설정하면서 상태가 유효한지까지 확인하려는 것일까? 
         전혀 알 수 없다.)

> 프로퍼티를 사용함으로써 위와 같은 혼동을 피할 수 있다.

        @property 데코레이터 : 무언가에 "응답을 위한 쿼리"
        @<property_name>.setter : 무언가를 "설정하기 위한 커맨드" (즉, 무언가의 상태를 변경하는 커맨드)

> 추가 조언 : **한 메서드에서 한 가지 이상의 일을 하지 말라!** 무언가 할당하고 유효성 검사를 하고 싶으면 두 개 이상의 문장으로 나눠야한다.